In [1]:
import pandas as pd # for data manipulation
print('pandas: %s' % pd.__version__) # print version
import numpy as np # for data manipulation
print('numpy: %s' % np.__version__) # print version
import math # to help with data reshaping of the data

import numpy as np
import torch
#import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pandas: 1.4.1
numpy: 1.21.5


In [2]:
df=pd.read_csv('weatherAUS.csv', encoding='utf-8')

# Drop records where target MinTemp=NaN or MaxTemp=NaN
df=df[pd.isnull(df['MinTemp'])==False]
df=df[pd.isnull(df['MaxTemp'])==False]

df['MedTemp']=df[['MinTemp', 'MaxTemp']].median(axis=1)

df2 = df.loc[df.Location=='Canberra'].loc[:, ['Date', 'MedTemp']]
df2.head()

,Date,MedTemp
45587,2007-11-01,16.15
45588,2007-11-02,20.45
45589,2007-11-03,18.55
45590,2007-11-04,14.40
45591,2007-11-05,11.85


In [3]:
from sklearn.model_selection import train_test_split

In [4]:
from sklearn.model_selection import train_test_split

def prep_data(datain, time_step):
    # 1. y-array  
    # First, create an array with indices for y elements based on the chosen time_step
    y_indices = np.arange(start=time_step, stop=len(datain), step=time_step)
    # Create y array based on the above indices 
    y_tmp = datain[y_indices]
    
    # 2. X-array  
    # We want to have the same number of rows for X as we do for y
    rows_X = len(y_tmp)
    # Since the last element in y_tmp may not be the last element of the datain, 
    # let's ensure that X array stops with the last y
    X_tmp = datain[range(time_step*rows_X)]
    # Now take this array and reshape it into the desired shape
    X_tmp = np.reshape(X_tmp, (rows_X, time_step, 1))
    return X_tmp, y_tmp

datain = np.array(df2.MedTemp)
X, Y = prep_data(datain, 7)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

In [5]:
X_train.shape
Y_train.shape
X_test.shape

(391, 7, 1)

(391,)

(98, 7, 1)

In [6]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size

        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

# model = nn.Sequential()
#model.add_module(nn.Linear(in_features=7, ))
# model.add_module("rnn", module=RNN(input_size=1, hidden_size=10, output_size=1))
rnn = RNN(input_size=1, hidden_size=10, output_size=1)


In [7]:
X_train.shape
X_train[:, 2, :].reshape(-1,1).shape



(391, 7, 1)

(391, 1)

In [8]:
import tqdm
criterion = nn.MSELoss()
learning_rate=0.001
losses = []

n_iter = 100

for i in tqdm.tqdm(range(n_iter)):
    hidden = torch.zeros(X_train.shape[0], 10).requires_grad_()
    bs = 10
    for i in range(X_train.shape[1]):
        indata = X_train[:, i, :].reshape(-1, 1)
        output, hidden = rnn(torch.tensor(indata).float(), hidden)

    loss = criterion(output.reshape(-1), torch.tensor(Y_train).float())
    loss.backward()
    for p in rnn.parameters():
            p.data.add_(p.grad.data, alpha=-learning_rate)

    losses.append(loss.item())

  0%|          | 0/100 [00:00<?, ?it/s]

tensor([[-0.2968, -0.2915,  0.0869,  0.1592,  0.1812, -0.0021, -0.1693,  0.1751,
          0.1080,  0.1299, -0.2643],
        [ 0.1884,  0.0022,  0.0952,  0.0136,  0.0231, -0.2148,  0.0246,  0.0089,
          0.2051,  0.1051, -0.1089],
        [ 0.2252, -0.1295,  0.1583,  0.0784,  0.1824,  0.1330, -0.2902, -0.1769,
         -0.0543,  0.2126,  0.0899],
        [-0.0276,  0.2408, -0.1443,  0.0998, -0.1958,  0.2141,  0.1592,  0.1314,
         -0.2212,  0.1721,  0.1183],
        [ 0.2217,  0.0369, -0.0783,  0.0842,  0.2872,  0.1347, -0.0702,  0.2764,
         -0.1199, -0.1336, -0.0095],
        [-0.1696,  0.0782, -0.0900, -0.2360,  0.2388,  0.2767,  0.1691,  0.0232,
         -0.1498,  0.0980, -0.1898],
        [ 0.0150, -0.2474, -0.1413,  0.2599, -0.0545, -0.2078, -0.0967, -0.1602,
          0.2482,  0.0259,  0.0149],
        [-0.0332,  0.1948, -0.0025,  0.1056,  0.2179, -0.2723, -0.1507, -0.0589,
          0.0004, -0.2099, -0.0987],
        [ 0.0118,  0.2706,  0.0567,  0.0322,  0.1469,  0

tensor([-0.1583, -0.0038, -0.2478, -0.0732, -0.0288,  0.2952,  0.0645,  0.1414,
         0.0987, -0.2968])

tensor([[-0.0415,  0.1793,  0.1504, -0.2458,  0.0856, -0.1267,  0.1473, -0.0204,
         -0.1017, -0.1531,  0.2974]])

tensor([-0.1226])

...

tensor([[-0.2968, -0.2915,  0.0869,  0.1592,  0.1812, -0.0021, -0.1693,  0.1751,
          0.1080,  0.1299, -0.2643],
        [ 0.1884,  0.0022,  0.0952,  0.0136,  0.0231, -0.2148,  0.0246,  0.0089,
          0.2051,  0.1051, -0.1089],
        [ 0.2252, -0.1295,  0.1583,  0.0784,  0.1824,  0.1330, -0.2902, -0.1769,
         -0.0543,  0.2126,  0.0899],
        [-0.0276,  0.2408, -0.1443,  0.0998, -0.1958,  0.2141,  0.1592,  0.1314,
         -0.2212,  0.1721,  0.1183],
        [ 0.2217,  0.0369, -0.0783,  0.0842,  0.2872,  0.1347, -0.0702,  0.2764,
         -0.1199, -0.1336, -0.0095],
        [-0.1696,  0.0782, -0.0900, -0.2360,  0.2388,  0.2767,  0.1691,  0.0232,
         -0.1498,  0.0980, -0.1898],
        [ 0.0150, -0.2474, -0.1413,  0.2599, -0.0545, -0.2078, -0.0967, -0.1602,
          0.2482,  0.0259,  0.0149],
        [-0.0332,  0.1948, -0.0025,  0.1056,  0.2179, -0.2723, -0.1507, -0.0589,
          0.0004, -0.2099, -0.0987],
        [ 0.0118,  0.2706,  0.0567,  0.0322,  0.1469,  0

tensor([-0.1583, -0.0038, -0.2478, -0.0732, -0.0288,  0.2952,  0.0645,  0.1414,
         0.0987, -0.2968])

tensor([[-0.0415,  0.1793,  0.1504, -0.2458,  0.0856, -0.1267,  0.1473, -0.0204,
         -0.1017, -0.1531,  0.2974]])

tensor([-0.1226])

100%|██████████| 100/100 [00:04<00:00, 23.63it/s]


In [9]:
losses

[232.59097290039062,
 232.59097290039062,
 232.59097290039062,
 232.59097290039062,
 232.59097290039062,
 232.59097290039062,
 232.59097290039062,
 232.59097290039062,
 232.59097290039062,
 232.59097290039062,
 232.59097290039062,
 232.59097290039062,
 232.59097290039062,
 232.59097290039062,
 232.59097290039062,
 232.59097290039062,
 232.59097290039062,
 232.59097290039062,
 232.59097290039062,
 232.59097290039062,
 232.59097290039062,
 232.59097290039062,
 232.59097290039062,
 232.59097290039062,
 232.59097290039062,
 232.59097290039062,
 232.59097290039062,
 232.59097290039062,
 232.59097290039062,
 232.59097290039062,
 232.59097290039062,
 232.59097290039062,
 232.59097290039062,
 232.59097290039062,
 232.59097290039062,
 232.59097290039062,
 232.59097290039062,
 232.59097290039062,
 232.59097290039062,
 232.59097290039062,
 232.59097290039062,
 232.59097290039062,
 232.59097290039062,
 232.59097290039062,
 232.59097290039062,
 232.59097290039062,
 232.59097290039062,
 232.59097290

In [42]:
output[:4]
Y_train[:4]

tensor([[0.],
        [0.],
        [0.],
        [0.]], grad_fn=<SliceBackward0>)

array([ 2.95,  3.65,  8.6 , 20.1 ])